In [1]:
import pandas as pd
import csv, io, os, psycopg2
from sqlalchemy import create_engine
from dotenv import load_dotenv

In [2]:
loc_df = pd.read_parquet('./taxi_zones.parquet')
loc_df.head()

,zone,borough,long,lat
LocationID,,,,
1,Newark Airport,EWR,-74.174000,40.691831
2,Jamaica Bay,Queens,-73.831299,40.616745
3,Allerton/Pelham Gardens,Bronx,-73.847422,40.864474
4,Alphabet City,Manhattan,-73.976968,40.723752
5,Arden Heights,Staten Island,-74.188484,40.552659


In [3]:
pickup_location_dim = loc_df.copy(deep=True)
pickup_location_dim.rename(columns={"LocationID":"pickup_location_id","long":"pickup_longitude","lat":"pickup_latitude","zone":"pickup_zone","borough":"pickup_borough"},inplace=True)
pickup_location_dim.head()

,pickup_zone,pickup_borough,pickup_longitude,pickup_latitude
LocationID,,,,
1,Newark Airport,EWR,-74.174000,40.691831
2,Jamaica Bay,Queens,-73.831299,40.616745
3,Allerton/Pelham Gardens,Bronx,-73.847422,40.864474
4,Alphabet City,Manhattan,-73.976968,40.723752
5,Arden Heights,Staten Island,-74.188484,40.552659


In [4]:
dropoff_location_dim = loc_df.copy(deep=True)
dropoff_location_dim.rename(columns={"LocationID":"dropoff_location_id","long":"dropoff_longitude","lat":"dropoff_latitude","zone":"dropoff_zone","borough":"dropoff_borough"},inplace=True)
dropoff_location_dim.head()

,dropoff_zone,dropoff_borough,dropoff_longitude,dropoff_latitude
LocationID,,,,
1,Newark Airport,EWR,-74.174000,40.691831
2,Jamaica Bay,Queens,-73.831299,40.616745
3,Allerton/Pelham Gardens,Bronx,-73.847422,40.864474
4,Alphabet City,Manhattan,-73.976968,40.723752
5,Arden Heights,Staten Island,-74.188484,40.552659


In [5]:
## Creating the vendor_dim table
vendor = [
    [1, 'Creative Mobile Technologies, LLC'],
    [2, 'VeriFone Inc.'],
]

vendor_dim = pd.DataFrame(vendor, columns=['vendor_id','vendor_name'])
vendor_dim

,vendor_id,vendor_name
0,1,"Creative Mobile Technologies, LLC"
1,2,VeriFone Inc.


In [6]:
## Creating the payment_type_dim table
payment = [
    [1, 'Credit card'],
    [2, 'Cash'],
    [3, 'No charge'],
    [4, 'Dispute'],
    [5, 'Unknown'],
    [6, 'Voided trip'],
]

payment_type_dim = pd.DataFrame(payment, columns=['payment_type_id', 'payment_type'])
payment_type_dim     

,payment_type_id,payment_type
0,1,Credit card
1,2,Cash
2,3,No charge
3,4,Dispute
4,5,Unknown
5,6,Voided trip


In [7]:
# Creating the rate_code_dim table
rate = {
    'rate_code_id': [1, 2, 3, 4, 5, 6],
    'rate': ['Standard rate', 'JFK', 'Newark', 'Nassau or Westchester', 'Negotiated fare', 'Group ride']
}

rate_code_dim = pd.DataFrame(rate)
rate_code_dim

,rate_code_id,rate
0,1,Standard rate
1,2,JFK
2,3,Newark
3,4,Nassau or Westchester
4,5,Negotiated fare
5,6,Group ride


In [12]:
#loading dotenv variables
load_dotenv()

host = os.getenv("HOST")
username = os.getenv("DBUSER")
password = os.getenv("PASSWORD")
database = os.getenv("DATABASE")

In [13]:
def create_tables():
    commands = (
    """
        CREATE TABLE IF NOT EXISTS vendor_dim (
            vendor_id integer not null primary key,
            vendor_name varchar(45) not null
        )
    """,
    """
        CREATE TABLE IF NOT EXISTS payment_type_dim (
            payment_type_id integer not null primary key,
            payment_type varchar(45) not null
        )
    """,
    """
        CREATE TABLE IF NOT EXISTS rate_code_dim (
            rate_code_id integer not null primary key,
            rate varchar(45) not null
        )
    """,
    """
        CREATE TABLE IF NOT EXISTS pickup_location_dim (
            pickup_location_id integer not null primary key,
            pickup_zone varchar(45) not null,
            pickup_borough varchar(45) not null,
            pickup_longitude float not null,
            pickup_latitude float not null
        )
    """,
    """
        CREATE TABLE IF NOT EXISTS dropoff_location_dim (
            dropoff_location_id integer not null primary key,
            dropoff_zone varchar(45) not null,
            dropoff_borough varchar(45) not null,
            dropoff_longitude float not null,
            dropoff_latitude float not null
        )
    """)

    try:
        conn = psycopg2.connect(
            host = host,
            database = database,
            user = username,
            password = password
        )

        cur = conn.cursor()
        for command in commands:
            cur.execute(command)

        cur.close()
        conn.commit()
    except (Exception) as e:
        print(e)
    finally:
        if conn is not None:
            conn.close()

create_tables()

In [15]:
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}/{database}')

def load_df_to_db(df, engine, table_name):
    df.head(0).to_sql(table_name, engine, if_exists='replace',index=False)
    conn = engine.raw_connection()
    
    cur = conn.cursor()
    output = io.StringIO()
    df.to_csv(output, sep='\t', header=False, index=False)
    output.seek(0)
    contents = output.getvalue()
    cur.copy_from(output, table_name, null="") # null values become ''
    conn.commit()
    cur.close()
    conn.close()


In [16]:
load_df_to_db(vendor_dim, engine, 'vendor_dim')
load_df_to_db(payment_type_dim, engine, 'payment_type_dim')
load_df_to_db(rate_code_dim, engine, 'rate_code_dim')
load_df_to_db(pickup_location_dim, engine, 'pickup_location_dim')
load_df_to_db(dropoff_location_dim, engine, 'dropoff_location_dim')